# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv('../output_data/Cities.csv')
city_data.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Prince Rupert,54.3161,-130.3201,49.69,72,100,5.75,CA,1635103817
1,Rikitea,-23.1203,-134.9692,74.70,72,23,17.83,PF,1635103919
2,Tuktoyaktuk,69.4541,-133.0374,30.20,93,75,2.30,CA,1635103920
3,Ushuaia,-54.8000,-68.3000,33.46,80,75,39.12,AR,1635103920
4,Raja,8.4596,25.6780,73.99,85,59,2.80,SS,1635103921


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
#configure gmaps
gmaps.configure(api_key=g_key)

#get the locations and humidity
city_loc = city_data[["Lat","Lng"]]
humidity = city_data["Humidity"]

layout = {
        'width': '1000px',
        'height': '600px',
        'border': '1px solid black',
        'padding': '1px'
}

#create a gmap figure
fig = gmaps.figure(map_type='HYBRID',layout = layout)

#add the heatmap layer
heat_layer = gmaps.heatmap_layer(city_loc, weights=humidity, dissipating=False, max_intensity=100, point_radius=1)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
weather = city_data.loc[(city_data['Max Temp'] >= 70) & (city_data['Max Temp'] <= 80) &
                       (city_data['Wind Speed'] <= 20) & (city_data['Humidity'] <= 30)]
weather

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
20,Bagé,-31.3314,-54.1069,71.38,24,0,10.36,BR,1635103925
177,Mar del Plata,-38.0023,-57.5575,71.62,28,0,18.41,AR,1635103795
209,Concepción del Oro,24.6333,-101.4167,73.47,24,99,4.07,MX,1635103975
283,Gobabis,-22.4500,18.9667,71.82,12,0,3.83,NaN,1635103745
386,Molina,-35.1167,-71.2833,79.38,29,0,12.66,CL,1635104025
509,Jalu,29.0331,21.5482,76.78,28,33,10.29,LY,1635103904


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = weather
hotel_df.reset_index(drop = True, inplace=True)
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bagé,-31.3314,-54.1069,71.38,24,0,10.36,BR,1635103925
1,Mar del Plata,-38.0023,-57.5575,71.62,28,0,18.41,AR,1635103795
2,Concepción del Oro,24.6333,-101.4167,73.47,24,99,4.07,MX,1635103975
3,Gobabis,-22.4500,18.9667,71.82,12,0,3.83,NaN,1635103745
4,Molina,-35.1167,-71.2833,79.38,29,0,12.66,CL,1635104025
5,Jalu,29.0331,21.5482,76.78,28,33,10.29,LY,1635103904


In [6]:
hotel_df = weather
# Adding a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

#parameters dic to udpate each iterations
params = {
        "raidus": 5000,
        #"rankby": "prominence",
        "types": "hotel",
        "keyword": "hotels",
        "key": g_key,
}


#use iterrows to iterate through dataframe.
for index, row in hotel_df.iterrows():
        #get lat, lng from df
        lat = row['Lat']
        lng = row['Lng']
        city = row['City']
        params["location"] = f'{lat},{lng}'
        
        hotel_data = requests.get(base_url, params=params).json()
        
        # url and make api request.
       # response = requests.get(base_url, params=params).json()
       # print(f"Retrieving Reuslts for Index {index}: {row['City']}.")
       # response = requests.get(base_url, params=params).json()
        #results
       # results = response['results']
        
        
        try:
            hotel_df.loc[row_indexer, "Hotel Name"] = hotel_data['results'][0]['name']
        
        except: 
            print('Issues with finding a hotel at', row['City'])
            hotel_df.loc[index, "Hotel Name"] = "NA"
    

hotel_df
    
        
        

<ipython-input-6-05e899fbc506>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"] = ""
/Users/prajesh/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Issues with finding a hotel at Bagé
Issues with finding a hotel at Mar del Plata
Issues with finding a hotel at Concepción del Oro
Issues with finding a hotel at Gobabis
Issues with finding a hotel at Molina
Issues with finding a hotel at Jalu


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Bagé,-31.3314,-54.1069,71.38,24,0,10.36,BR,1635103925,NA
1,Mar del Plata,-38.0023,-57.5575,71.62,28,0,18.41,AR,1635103795,NA
2,Concepción del Oro,24.6333,-101.4167,73.47,24,99,4.07,MX,1635103975,NA
3,Gobabis,-22.4500,18.9667,71.82,12,0,3.83,NaN,1635103745,NA
4,Molina,-35.1167,-71.2833,79.38,29,0,12.66,CL,1635104025,NA
5,Jalu,29.0331,21.5482,76.78,28,33,10.29,LY,1635103904,NA


In [7]:
hotel_df = hotel_df.rename(columns={"Latitude": "Lat", "Longitude": "Lng"})
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,Bagé,-31.3314,-54.1069,71.38,24,0,10.36,BR,1635103925,NA
1,Mar del Plata,-38.0023,-57.5575,71.62,28,0,18.41,AR,1635103795,NA
2,Concepción del Oro,24.6333,-101.4167,73.47,24,99,4.07,MX,1635103975,NA
3,Gobabis,-22.4500,18.9667,71.82,12,0,3.83,NaN,1635103745,NA
4,Molina,-35.1167,-71.2833,79.38,29,0,12.66,CL,1635104025,NA
5,Jalu,29.0331,21.5482,76.78,28,33,10.29,LY,1635103904,NA


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))